In [1]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from random import shuffle
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
patht="D://datasets//MURA//train//XR_FINGER//True";
pathf="D://datasets//MURA//train//XR_FINGER//False";
path_tt="D://datasets//MURA//valid//XR_FINGER//False";
path_tf="D://datasets//MURA//valid//XR_FINGER//TRUE";

In [3]:
def create_label(image_name):
    if image_name=='True':
        return np.array([1,0]);
    else:
        return np.array([0,1]);   

In [4]:
def create_train_image():
    training_data=[];
    for img1 in tqdm(os.listdir(patht)):
        path=os.path.join(patht,img1);
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        training_data.append([np.array(img_data),create_label('True')])
    for img1 in tqdm(os.listdir(pathf)):
        path=os.path.join(pathf,img1)
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        training_data.append([np.array(img_data),create_label('False')]);
    shuffle(training_data)
    np.save('train_data.npy',training_data);
    return training_data

In [5]:
def create_test_data():
    testing_data=[];img_num=0
    for img in tqdm(os.listdir(path_tt)):
        path=os.path.join(path_tt,img);
        img_num=img_num+1;
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        testing_data.append([np.array(img_data),img_num])
    for img in tqdm(os.listdir(path_tf)):
        path=os.path.join(path_tf,img);
        img_num=img_num+1;
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        testing_data.append([np.array(img_data),img_num])
    shuffle(testing_data);
    np.save('test_data.npy',testing_data);
    return testing_data

In [6]:
train_data=create_train_image();
train=train_data[:-800];
test=train_data[-800:];
X_train=np.array([i[0] for i in train]).reshape(-1,50,50,1)
y_train=np.array([i[1] for i in train])
X_test=np.array([i[0] for i in test]).reshape(-1,50,50,1);
y_test=[i[1] for i in test]

100%|█████████████████████████████████████████████████████████████████████████████| 3138/3138 [00:15<00:00, 198.44it/s]


In [7]:
convnet=input_data(shape=[None,50,50,1],name='input')
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)#1
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)#2
convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)#3

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [8]:
def conv(convnet,steps):
    for i in range(steps):
        convnet=conv_2d(convnet,256,5,activation='relu')
        convnet=max_pool_2d(convnet,5)#67
    return convnet;

In [9]:
convnet=conv(convnet,1)
convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)#68
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)#69
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)#70
convnet=fully_connected(convnet,1024,activation='relu')
convnet=dropout(convnet,0.8)
convnet=fully_connected(convnet,2,activation='softmax')
convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy',name='complete')
model=tflearn.DNN(convnet,tensorboard_dir='log')
model.fit(X_train,y_train,n_epoch=50,validation_set=(X_test,y_test),snapshot_step=500,show_metric=True,run_id='Finger')
save_path="D://datasets//Philips//XR_FINGER.h5"


Training Step: 3399  | total loss: 0.02602 | time: 3.441s
| Adam | epoch: 050 | loss: 0.02602 - acc: 0.9888 -- iter: 4288/4306
Training Step: 3400  | total loss: 0.02497 | time: 4.479s
| Adam | epoch: 050 | loss: 0.02497 - acc: 0.9899 | val_loss: 2.03734 - val_acc: 0.6925 -- iter: 4306/4306
--


In [19]:
model.save('D://Philips//my_model.tflearn')

INFO:tensorflow:C:\Users\Debasis\my_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [20]:
model.load('my_model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Debasis\my_model.tflearn


AttributeError: 'DNN' object has no attribute 'get_config'

In [16]:
saver=tf.train.Saver()
saver.save(model,'my_model')

TypeError: 'sess' must be a Session; <tflearn.models.dnn.DNN object at 0x0000024925653470>

In [21]:
test_data=create_test_data();

100%|███████████████████████████████████████████████████████████████████████████████| 247/247 [00:00<00:00, 636.66it/s]


In [15]:
from tensorflow import keras
from keras.models import load_model

Using TensorFlow backend.


In [23]:
model.load("D://datasets//Philips//XR_FINGER.tfl")

ValueError: The passed save_path is not a valid checkpoint: D://datasets//Philips//XR_FINGER.tfl

In [27]:
'''test_data=create_test_data();
for num,data in enumerate(test_data[:16]):
    img_num=data[1]
    img_data=data[0]
    orig=img_data
    data=img_data.reshape(50,50,1)
    model_out=model.predict([data])[0]
    if np.argmax(model_out)==1:
        str_label='Fracture'
    else:
        str_label='Not Fracture'
    plt.imshow(orig,cmap='gray')
    plt.title(str_label)
    plt.gca().set_axis_off()
    plt.show()'''
orig=cv2.imread("C://Users//Debasis//Downloads//t4.jpg")
orig=
data=cv2.resize(orig,(50,50))
model_out=model.predict(data)
if np.argmax(model_out)==1:
    str_label='Fracture'
else:
    str_label='Not Fracture'
plt.imshow(orig,cmap='gray')
plt.title(str_label)
plt.gca().set_axis_off()
plt.show()


ValueError: Cannot feed value of shape (50, 50, 3) for Tensor 'input/X:0', which has shape '(?, 50, 50, 1)'